In [1]:
import pandas as pd
from pandas import json_normalize
import google.generativeai as genai
from google.api_core import retry
from sklearn.model_selection import train_test_split

import dotenv
import os
import json
import tldextract

In [2]:
# Turn json into pandas dataframe
data = pd.read_json('large_eve.json', lines=True)
data

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,...,vars,flow,icmp_type,icmp_code,tcp,smtp,email,app_proto_tc,app_proto_ts,stats
0,2017-07-22T17:33:15.784100-0500,2.835707e+14,18519.0,alert,130.0,192.168.203.200,2328.0,192.181.145.109,445.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-07-22T17:33:16.571273-0500,1.519048e+15,21531.0,ssh,140.0,192.168.204.60,62990.0,192.168.26.254,22.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-07-22T17:33:16.661646-0500,1.327836e+15,22269.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680.0,2001:0500:0001:0000:0000:0000:803f:0235,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-07-22T17:33:17.561063-0500,9.632021e+14,28731.0,http,140.0,192.168.204.59,38011.0,192.168.24.201,80.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-07-22T17:33:17.595416-0500,9.632021e+14,28994.0,fileinfo,140.0,192.168.24.201,80.0,192.168.204.59,38011.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746904,2017-07-22T19:47:49.301548-0500,1.970284e+15,NaN,flow,NaN,192.168.203.200,2007.0,192.183.45.157,445.0,TCP,...,NaN,"{'pkts_toserver': 2, 'pkts_toclient': 0, 'byte...",NaN,NaN,"{'tcp_flags': '02', 'tcp_flags_ts': '02', 'tcp...",NaN,NaN,NaN,NaN,NaN
746905,2017-07-22T19:47:49.301548-0500,2.251792e+15,NaN,flow,NaN,192.168.198.57,52498.0,192.168.21.141,22.0,TCP,...,NaN,"{'pkts_toserver': 2, 'pkts_toclient': 0, 'byte...",NaN,NaN,"{'tcp_flags': '02', 'tcp_flags_ts': '02', 'tcp...",NaN,NaN,NaN,NaN,NaN
746906,2017-07-22T19:47:49.301548-0500,8.444188e+14,NaN,flow,NaN,192.168.201.72,57805.0,192.168.26.225,80.0,TCP,...,NaN,"{'pkts_toserver': 6, 'pkts_toclient': 6, 'byte...",NaN,NaN,"{'tcp_flags': '1b', 'tcp_flags_ts': '1b', 'tcp...",NaN,NaN,NaN,NaN,NaN
746907,2017-07-22T19:47:49.301548-0500,2.814688e+14,NaN,flow,NaN,192.168.201.72,43004.0,192.168.26.225,80.0,TCP,...,NaN,"{'pkts_toserver': 6, 'pkts_toclient': 6, 'byte...",NaN,NaN,"{'tcp_flags': '1b', 'tcp_flags_ts': '1b', 'tcp...",NaN,NaN,NaN,NaN,NaN


In [3]:
print("Records amount: ", len(data))

Records amount:  746909


In [4]:
data

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,...,vars,flow,icmp_type,icmp_code,tcp,smtp,email,app_proto_tc,app_proto_ts,stats
0,2017-07-22T17:33:15.784100-0500,2.835707e+14,18519.0,alert,130.0,192.168.203.200,2328.0,192.181.145.109,445.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-07-22T17:33:16.571273-0500,1.519048e+15,21531.0,ssh,140.0,192.168.204.60,62990.0,192.168.26.254,22.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-07-22T17:33:16.661646-0500,1.327836e+15,22269.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680.0,2001:0500:0001:0000:0000:0000:803f:0235,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-07-22T17:33:17.561063-0500,9.632021e+14,28731.0,http,140.0,192.168.204.59,38011.0,192.168.24.201,80.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-07-22T17:33:17.595416-0500,9.632021e+14,28994.0,fileinfo,140.0,192.168.24.201,80.0,192.168.204.59,38011.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746904,2017-07-22T19:47:49.301548-0500,1.970284e+15,NaN,flow,NaN,192.168.203.200,2007.0,192.183.45.157,445.0,TCP,...,NaN,"{'pkts_toserver': 2, 'pkts_toclient': 0, 'byte...",NaN,NaN,"{'tcp_flags': '02', 'tcp_flags_ts': '02', 'tcp...",NaN,NaN,NaN,NaN,NaN
746905,2017-07-22T19:47:49.301548-0500,2.251792e+15,NaN,flow,NaN,192.168.198.57,52498.0,192.168.21.141,22.0,TCP,...,NaN,"{'pkts_toserver': 2, 'pkts_toclient': 0, 'byte...",NaN,NaN,"{'tcp_flags': '02', 'tcp_flags_ts': '02', 'tcp...",NaN,NaN,NaN,NaN,NaN
746906,2017-07-22T19:47:49.301548-0500,8.444188e+14,NaN,flow,NaN,192.168.201.72,57805.0,192.168.26.225,80.0,TCP,...,NaN,"{'pkts_toserver': 6, 'pkts_toclient': 6, 'byte...",NaN,NaN,"{'tcp_flags': '1b', 'tcp_flags_ts': '1b', 'tcp...",NaN,NaN,NaN,NaN,NaN
746907,2017-07-22T19:47:49.301548-0500,2.814688e+14,NaN,flow,NaN,192.168.201.72,43004.0,192.168.26.225,80.0,TCP,...,NaN,"{'pkts_toserver': 6, 'pkts_toclient': 6, 'byte...",NaN,NaN,"{'tcp_flags': '1b', 'tcp_flags_ts': '1b', 'tcp...",NaN,NaN,NaN,NaN,NaN


## Part 1

Get all DNS records

In [5]:
# Filter only dns events
data = data[data['event_type'] == "dns"]

In [6]:
len(data)

15749

In [7]:
data.head()

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,...,vars,flow,icmp_type,icmp_code,tcp,smtp,email,app_proto_tc,app_proto_ts,stats
2,2017-07-22T17:33:16.661646-0500,1.327836e+15,22269.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680.0,2001:0500:0001:0000:0000:0000:803f:0235,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,2017-07-22T17:33:24.990320-0500,2.022925e+15,54352.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051.0,2001:0500:0003:0000:0000:0000:0000:0042,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,2017-07-22T17:33:27.379891-0500,5.785448e+14,54519.0,dns,150.0,192.168.205.170,31393.0,192.168.207.4,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,2017-07-22T17:33:27.380146-0500,5.785448e+14,54520.0,dns,150.0,192.168.207.4,53.0,192.168.205.170,31393.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,2017-07-22T17:33:27.380146-0500,5.785448e+14,54520.0,dns,150.0,192.168.207.4,53.0,192.168.205.170,31393.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
data.shape

(15749, 28)

Filter all domains that are type A

In [9]:
# Normalize the json data inside dns column
dns_expanded = json_normalize(data['dns'])
dns_expanded = dns_expanded.drop('tx_id', axis=1)

In [10]:
# Join it with the original dataframe
data = data.join(dns_expanded)
data.head()

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,...,app_proto_tc,app_proto_ts,stats,type,id,rrname,rrtype,rcode,ttl,rdata
2,2017-07-22T17:33:16.661646-0500,1.327836e+15,22269.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680.0,2001:0500:0001:0000:0000:0000:803f:0235,53.0,UDP,...,NaN,NaN,NaN,query,54724.0,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,NaN,NaN,NaN
38,2017-07-22T17:33:24.990320-0500,2.022925e+15,54352.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051.0,2001:0500:0003:0000:0000:0000:0000:0042,53.0,UDP,...,NaN,NaN,NaN,query,52311.0,api.wunderground.com,A,NaN,NaN,NaN
39,2017-07-22T17:33:27.379891-0500,5.785448e+14,54519.0,dns,150.0,192.168.205.170,31393.0,192.168.207.4,53.0,UDP,...,NaN,NaN,NaN,query,22415.0,aolmtcmxm03.office.aol.com.ad.aol.aoltw.net,A,NaN,NaN,NaN
40,2017-07-22T17:33:27.380146-0500,5.785448e+14,54520.0,dns,150.0,192.168.207.4,53.0,192.168.205.170,31393.0,UDP,...,NaN,NaN,NaN,answer,22415.0,aolmtcmxm03.office.aol.com.ad.aol.aoltw.net,NaN,NXDOMAIN,NaN,NaN
41,2017-07-22T17:33:27.380146-0500,5.785448e+14,54520.0,dns,150.0,192.168.207.4,53.0,192.168.205.170,31393.0,UDP,...,NaN,NaN,NaN,answer,22415.0,<root>,SOA,NXDOMAIN,20864.0,NaN


In [11]:
data.shape

(15749, 35)

In [12]:
# Filter only type A records
data = data[data['rrtype'] == 'A']
data

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,...,app_proto_tc,app_proto_ts,stats,type,id,rrname,rrtype,rcode,ttl,rdata
2,2017-07-22T17:33:16.661646-0500,1.327836e+15,22269.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680.0,2001:0500:0001:0000:0000:0000:803f:0235,53.0,UDP,...,NaN,NaN,NaN,query,54724.0,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,NaN,NaN,NaN
38,2017-07-22T17:33:24.990320-0500,2.022925e+15,54352.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051.0,2001:0500:0003:0000:0000:0000:0000:0042,53.0,UDP,...,NaN,NaN,NaN,query,52311.0,api.wunderground.com,A,NaN,NaN,NaN
39,2017-07-22T17:33:27.379891-0500,5.785448e+14,54519.0,dns,150.0,192.168.205.170,31393.0,192.168.207.4,53.0,UDP,...,NaN,NaN,NaN,query,22415.0,aolmtcmxm03.office.aol.com.ad.aol.aoltw.net,A,NaN,NaN,NaN
45,2017-07-22T17:33:38.537676-0500,2.167545e+15,55688.0,dns,180.0,192.168.207.4,53.0,192.168.198.62,35092.0,UDP,...,NaN,NaN,NaN,query,27069.0,safebrowsing.clients.google.com,A,NaN,NaN,NaN
54,2017-07-22T17:33:26.875982-0500,1.936682e+15,54496.0,dns,150.0,192.168.207.4,53.0,192.168.205.188,46250.0,UDP,...,NaN,NaN,NaN,query,35974.0,stork79.dropbox.com,A,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11336,2017-07-22T17:44:50.629199-0500,1.982050e+15,125660.0,dns,130.0,192.168.203.75,47706.0,192.168.207.4,53.0,UDP,...,NaN,NaN,NaN,query,46432.0,mirror.clarkson.edu,A,NaN,NaN,NaN
11427,2017-07-22T17:45:15.766444-0500,9.112416e+14,131968.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,61474.0,2001:0500:002f:0000:0000:0000:0000:000f,53.0,UDP,...,NaN,NaN,NaN,query,40595.0,repo.genomics.upenn.edu,A,NaN,NaN,NaN
11931,2017-07-22T17:45:05.763382-0500,1.593102e+15,128354.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,24979.0,2001:07fd:0000:0000:0000:0000:0000:0001,53.0,UDP,...,NaN,NaN,NaN,query,27048.0,ftp.usf.edu,A,NaN,NaN,NaN
13242,2017-07-22T17:46:04.610794-0500,4.295309e+14,151993.0,dns,110.0,192.168.207.4,53.0,192.168.201.65,1034.0,UDP,...,NaN,NaN,NaN,query,31884.0,centos.cs.wisc.edu,A,NaN,NaN,NaN


Domain filter to find only unique domains records

In [13]:
# Find only records with unique domain names
data = data.drop_duplicates(subset=['rrname'])
data

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,...,app_proto_tc,app_proto_ts,stats,type,id,rrname,rrtype,rcode,ttl,rdata
2,2017-07-22T17:33:16.661646-0500,1.327836e+15,22269.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680.0,2001:0500:0001:0000:0000:0000:803f:0235,53.0,UDP,...,NaN,NaN,NaN,query,54724.0,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,NaN,NaN,NaN
38,2017-07-22T17:33:24.990320-0500,2.022925e+15,54352.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051.0,2001:0500:0003:0000:0000:0000:0000:0042,53.0,UDP,...,NaN,NaN,NaN,query,52311.0,api.wunderground.com,A,NaN,NaN,NaN
39,2017-07-22T17:33:27.379891-0500,5.785448e+14,54519.0,dns,150.0,192.168.205.170,31393.0,192.168.207.4,53.0,UDP,...,NaN,NaN,NaN,query,22415.0,aolmtcmxm03.office.aol.com.ad.aol.aoltw.net,A,NaN,NaN,NaN
45,2017-07-22T17:33:38.537676-0500,2.167545e+15,55688.0,dns,180.0,192.168.207.4,53.0,192.168.198.62,35092.0,UDP,...,NaN,NaN,NaN,query,27069.0,safebrowsing.clients.google.com,A,NaN,NaN,NaN
54,2017-07-22T17:33:26.875982-0500,1.936682e+15,54496.0,dns,150.0,192.168.207.4,53.0,192.168.205.188,46250.0,UDP,...,NaN,NaN,NaN,query,35974.0,stork79.dropbox.com,A,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8901,2017-07-22T17:40:52.624538-0500,7.628848e+14,97256.0,dns,150.0,192.168.205.58,33909.0,192.168.25.37,53.0,UDP,...,NaN,NaN,NaN,query,4675.0,versioncheck.addons.mozilla.org,A,NaN,NaN,NaN
9709,2017-07-22T17:40:07.996192-0500,1.950001e+15,91582.0,dns,110.0,192.168.201.57,57028.0,192.168.207.4,53.0,UDP,...,NaN,NaN,NaN,query,12629.0,www.msftncsi.com,A,NaN,NaN,NaN
10310,2017-07-22T17:44:23.774086-0500,5.230925e+14,122488.0,dns,180.0,192.168.207.4,53.0,192.168.198.62,39404.0,UDP,...,NaN,NaN,NaN,query,63932.0,wpad.home,A,NaN,NaN,NaN
11294,2017-07-22T17:42:05.548555-0500,1.344101e+15,105942.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,48079.0,2001:0500:0003:0000:0000:0000:0000:0042,53.0,UDP,...,NaN,NaN,NaN,query,56575.0,mirrors.xmission.com,A,NaN,NaN,NaN


In [14]:
len(data)

64

Extract the Top Level Domain (TLD) from eath domain

In [15]:
data['domain_tld'] = data['rrname'].apply(tldextract.extract).apply(lambda x: x.domain+'.'+x.suffix).drop_duplicates()

/var/folders/wx/3fwd1ycn21q0f_72nz8kjrr00000gn/T/ipykernel_62621/2559329416.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['domain_tld'] = data['rrname'].apply(tldextract.extract).apply(lambda x: x.domain+'.'+x.suffix).drop_duplicates()


In [16]:
domain_list = data["domain_tld"].dropna().astype(str).drop_duplicates().tolist()
print(len(domain_list))
print(domain_list)

53
['aoltw.net', 'wunderground.com', 'google.com', 'dropbox.com', 'mozilla.com', 'aol.com', 'vmware.com', 'informaction.com', 'offensive-security.com', 'stopbadware.org', 'microsoft.com', 'flickr.com', 'home.', 'localdomain.', 'cakephp.org', 'arrancar.org', 'adams.net', 'netriplex.com', 'sanctuaryhost.com', 'gigenet.com', 'tummy.com', 'liquidweb.com', 'usf.edu', 'rocketinternet.net', '5ninesolutions.com', 'ecvps.com', 'uidaho.edu', 'nyi.net', 'interworx.info', 'tds.net', 'wisc.edu', 'facebook.net', 'fastserv.com', 'easynews.com', 'clarkson.edu', 'stanford.edu', 'team-cymru.org', 'atlantic.net', 'rit.edu', '56".', 'pdx.edu', 'comcast.net', 'me.com', 'vpn.', 'threatspace.net', 'wpad.', 'windowsupdate.com', 'mac.com', 'stayonline.net', 'mozilla.org', 'msftncsi.com', 'xmission.com', 'upenn.edu']


## Part 2

In [17]:
dotenv.load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

In [18]:
retry_policy = {
    "retry": retry.Retry(predicate=retry.if_transient_error, initial=10, multiplier=1.5, timeout=200)
}

In [19]:
import enum

class Classification(enum.Enum):
    POSITIVE = "1"
    NEGATIVE = "0"

model = genai.GenerativeModel(
    'models/gemini-2.0-flash',
    generation_config=genai.GenerationConfig(
        temperature=0.95,
        top_p=0.95,
        top_k=80,
        response_mime_type="text/x.enum",
        response_schema=Classification
    ))

In [20]:
responses = dict()

In [21]:
# for i in range(0, len(domain_list), 10):
#     batch = domain_list[i:i+10]
#     for index, domain in enumerate(batch):
#         prompt = f"""Classify the following domain TLD as POSITIVE (algorithm generated) or NEGATIVE (legitimate):

#         TLD to classify: {domain}

#         Classification guidelines:
#         - LEGITIMATE: Standard recognized TLDs (com, org, net, edu, gov, io, co, app, etc.)
#         - DGA-GENERATED: Likely algorithmically generated TLDs showing these characteristics:
#         * Random-appearing letter/number combinations
#         * Unusual length (extremely short or long)
#         * Non-pronounceable character sequences
#         * Excessive digits mixed with letters

#         Examples:
#         - Legitimate: com, org, net, edu, gov, io, co, app, info, biz
#         - DGA-Generated: xyz123, a7b9, m5k2p, qw34rt, 5j8k2
#         """
#         response = model.generate_content(prompt, request_options=retry_policy)
#         print(response.text)
#         responses[domain] = response.text

In [22]:
# print(responses)
# print(len(responses))

In [23]:
with open('responses.json', 'r') as file:
    responses = json.load(file)

Separa los DGAs de los dominios legítimos

In [24]:
dga_domains = dict()

In [25]:
for item in responses.items():
    if item[1] == "1":
        dga_domains[item[0]] = item[1]

Guardamos los DGAs en un archivo json

In [26]:
with open("responses.json", "w") as f:
    json.dump(responses, f)

## Part 3

In [27]:
top_domains = pd.read_csv('top-1m.csv', header=None, names=['id', 'domain'])

In [28]:
len(top_domains)

1000000

In [29]:
for domain in dga_domains.items():
    print(domain[0])
    if domain[0] in top_domains['domain']:
        print("Domain deleted: ", domain[0])
        dga_domains.pop(domain)

wunderground.com
aoltw.net
110phpmyadmin.
ntkrnlpa.info
theanime.cn
ecvps.com
1201.
msftncsi.com
254.
ejfodfmfxlkgifuf.xyz


In [30]:
len(dga_domains)

10

In [31]:
dga_domains_keys = list(dga_domains.keys())

In [32]:
filtered_data = data[data['rrname'].str.endswith(tuple(dga_domains_keys))]
filtered_data

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,...,app_proto_ts,stats,type,id,rrname,rrtype,rcode,ttl,rdata,domain_tld
2,2017-07-22T17:33:16.661646-0500,1.327836e+15,22269.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680.0,2001:0500:0001:0000:0000:0000:803f:0235,53.0,UDP,...,NaN,NaN,query,54724.0,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,NaN,NaN,NaN,aoltw.net
38,2017-07-22T17:33:24.990320-0500,2.022925e+15,54352.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051.0,2001:0500:0003:0000:0000:0000:0000:0042,53.0,UDP,...,NaN,NaN,query,52311.0,api.wunderground.com,A,NaN,NaN,NaN,wunderground.com
39,2017-07-22T17:33:27.379891-0500,5.785448e+14,54519.0,dns,150.0,192.168.205.170,31393.0,192.168.207.4,53.0,UDP,...,NaN,NaN,query,22415.0,aolmtcmxm03.office.aol.com.ad.aol.aoltw.net,A,NaN,NaN,NaN,NaN
97,2017-07-22T17:33:26.874734-0500,2.973404e+14,54491.0,dns,150.0,192.168.205.188,44501.0,192.168.207.4,53.0,UDP,...,NaN,NaN,query,26728.0,aolmtcmxm04.office.aol.com.ad.aol.aoltw.net,A,NaN,NaN,NaN,NaN
5051,2017-07-22T17:41:00.683738-0500,2.133903e+15,98052.0,dns,150.0,192.168.205.58,33909.0,192.168.25.177,53.0,UDP,...,NaN,NaN,query,60854.0,mirrors.ecvps.com,A,NaN,NaN,NaN,ecvps.com
6934,2017-07-22T17:41:00.950115-0500,1.939023e+15,98136.0,dns,150.0,192.168.205.58,33909.0,192.168.25.192,53.0,UDP,...,NaN,NaN,query,10582.0,knoa-coll.ops.aol.com.ad.aol.aoltw.net,A,NaN,NaN,NaN,NaN
7927,2017-07-22T17:40:52.271174-0500,3.962922e+14,97200.0,dns,150.0,192.168.205.58,33909.0,192.168.25.4,53.0,UDP,...,NaN,NaN,query,57845.0,wpad.ad.aol.aoltw.net,A,NaN,NaN,NaN,NaN
7930,2017-07-22T17:40:52.542753-0500,1.076271e+15,97235.0,dns,150.0,192.168.205.58,33909.0,192.168.25.30,53.0,UDP,...,NaN,NaN,query,56280.0,wpad.aol.aoltw.net,A,NaN,NaN,NaN,NaN
9709,2017-07-22T17:40:07.996192-0500,1.950001e+15,91582.0,dns,110.0,192.168.201.57,57028.0,192.168.207.4,53.0,UDP,...,NaN,NaN,query,12629.0,www.msftncsi.com,A,NaN,NaN,NaN,msftncsi.com


In [35]:
filtered_data[['rrname', 'timestamp']]

,rrname,timestamp
2,hpca-tier2.office.aol.com.ad.aol.aoltw.net,2017-07-22T17:33:16.661646-0500
38,api.wunderground.com,2017-07-22T17:33:24.990320-0500
39,aolmtcmxm03.office.aol.com.ad.aol.aoltw.net,2017-07-22T17:33:27.379891-0500
97,aolmtcmxm04.office.aol.com.ad.aol.aoltw.net,2017-07-22T17:33:26.874734-0500
5051,mirrors.ecvps.com,2017-07-22T17:41:00.683738-0500
6934,knoa-coll.ops.aol.com.ad.aol.aoltw.net,2017-07-22T17:41:00.950115-0500
7927,wpad.ad.aol.aoltw.net,2017-07-22T17:40:52.271174-0500
7930,wpad.aol.aoltw.net,2017-07-22T17:40:52.542753-0500
9709,www.msftncsi.com,2017-07-22T17:40:07.996192-0500


Basandonos en el patrón que se utiliza normalmente para crear DGAs, podemos observar que el único que no podría cumplir con este patrón es el dominio `api.wunderground.com`. Por lo que no podemos concluir que este dominio sea creado por un algoritmo, sin embargo, con el resto se puede crear ciertas sospechas, bien fundamentadas, de que los demás 8 dominios son generados por el algoritmo. Esto porque los demás dominios no poseen nombres coherentes, e incluso, aunque algunos tienen un nombre coherente al inicio, se puede ver que el resto del url parece bastante aleatorio, como siguiendo un patrón. También se puede notar que tenemos bastante dominios que terminan con la última parte igual, solo con la primera parte cambiado, haciendo referencia a que estos fueron generados por el mismo algoritmo, que solo cambiaba la primera parte del dominio. 

Cabe resaltar que puede por usar el LLM, y posiblemente por el prompt utilizado, no se pudo obtener todos los dominios DGA (Como nos falto con el siguiente faltante `vtlfccmfxlkgifuf.com`), e incluso uno que era legítimo (Como con `api.wunderground.com`), por lo que los resultados pueden estar faltando de algunos dominios que no fueron detectados.